### Polish presidential elections 2015

I'm going to create some visualisations for the Polish presidential elections that happened in May 2015.

Data source: https://prezydent2015.pkw.gov.pl/319_Pierwsze_glosowanie.html <br> https://prezydent2015.pkw.gov.pl/325_Ponowne_glosowanie.html

### Importing necessary packages

In [1]:
import pandas
import geopandas
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

### Loading the data

In [2]:
df = pandas.read_excel(r"..\\data\\1st_round\\wyniki_tura1-1.xls")
df.head(30)

,Województwo,Gmina,TERYT gminy,Numer obwodu,Siedziba,Głosowanie rozpoczęło się w dniu,i trwało do godz,Liczba wyborców uprawnionych do głosowania,Komisja otrzymała kart do głosowania,Nie wykorzystano kart do głosowania,...,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,dolnośląskie,m. Bolesławiec,20101,1,"Miejskie Przedszkole Publiczne Nr 3, ul. Ceram...",10.05.2015 07:00:00,21:00:00,1542,1300,632,...,212,9,215,27,2,163,14,11,0,3
1,dolnośląskie,m. Bolesławiec,20101,2,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",10.05.2015 07:00:00,21:00:00,1369,1200,467,...,203,7,250,24,5,196,23,10,0,3
2,dolnośląskie,m. Bolesławiec,20101,3,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1510,1300,551,...,199,7,251,26,3,212,21,5,2,3
3,dolnośląskie,m. Bolesławiec,20101,4,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1523,1302,521,...,209,3,303,33,6,186,17,6,0,2
4,dolnośląskie,m. Bolesławiec,20101,5,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1492,1298,588,...,192,5,278,28,6,146,23,12,2,3
5,dolnośląskie,m. Bolesławiec,20101,6,"Miejski Zespół Szkół Nr 3, ul. Juliusza Słowac...",10.05.2015 07:00:00,21:00:00,1480,1301,731,...,134,5,247,20,2,126,16,5,1,0
6,dolnośląskie,m. Bolesławiec,20101,7,"Miejski Zespół Szkół Nr 2, ul. Jana Pawła II 3...",10.05.2015 07:00:00,21:00:00,1485,1302,497,...,248,3,354,17,3,123,29,13,0,5
7,dolnośląskie,m. Bolesławiec,20101,8,"Miejski Zespół Szkół Nr 2, ul. Jana Pawła II 3...",10.05.2015 07:00:00,21:00:00,1463,1300,445,...,272,9,331,25,3,165,24,15,1,1
8,dolnośląskie,m. Bolesławiec,20101,9,"Miejskie Przedszkole Publiczne Nr 6, ul. Piotr...",10.05.2015 07:00:00,21:00:00,1520,1301,598,...,205,4,283,23,6,148,16,9,0,1
9,dolnośląskie,m. Bolesławiec,20101,10,"Gimnazjum Samorządowe Nr 2, ul. Bielska 5, 59-...",10.05.2015 07:00:00,21:00:00,1560,1301,471,...,219,4,314,35,3,191,23,21,2,4


### Cleanup

First of all, I see that there's data for every single ward, which is a lot of data.

In [3]:
len(df.index)

27817

The dataset contains electoral data for nearly 28,000 wards that were created for the elections in the entire country. It's pretty neat, but I think I'll sum the data into municipalities for better presentation.

Secondly, there's too much unnecessary information. The start and end time was exactly the same in the vast majority of wards, and the data regarding the methods by which people voted is definitely too detailed. I'll remove the redundant columns.

However, there is one more thing to note. Each ward's municipality code is missing the leading zero. This could cause problems when pairing this data with geometry shape files. The problem is caused by Pandas treating the values as numbers, when they should be treated as text instead. I'll load the data again, this time with correct type.

In [4]:
df = pandas.read_excel(r"..\\data\\1st_round\\wyniki_tura1-1.xls", dtype={"TERYT gminy":object})
df.head()

,Województwo,Gmina,TERYT gminy,Numer obwodu,Siedziba,Głosowanie rozpoczęło się w dniu,i trwało do godz,Liczba wyborców uprawnionych do głosowania,Komisja otrzymała kart do głosowania,Nie wykorzystano kart do głosowania,...,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,dolnośląskie,m. Bolesławiec,020101,1,"Miejskie Przedszkole Publiczne Nr 3, ul. Ceram...",10.05.2015 07:00:00,21:00:00,1542,1300,632,...,212,9,215,27,2,163,14,11,0,3
1,dolnośląskie,m. Bolesławiec,020101,2,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",10.05.2015 07:00:00,21:00:00,1369,1200,467,...,203,7,250,24,5,196,23,10,0,3
2,dolnośląskie,m. Bolesławiec,020101,3,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1510,1300,551,...,199,7,251,26,3,212,21,5,2,3
3,dolnośląskie,m. Bolesławiec,020101,4,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1523,1302,521,...,209,3,303,33,6,186,17,6,0,2
4,dolnośląskie,m. Bolesławiec,020101,5,"Miejski Zespół Szkół Nr 1, ul. Dolne Młyny 60,...",10.05.2015 07:00:00,21:00:00,1492,1298,588,...,192,5,278,28,6,146,23,12,2,3


The leading zero is back. I'll keep the important columns and remove the rest.

In [5]:
df = df[['Województwo', 'Gmina', 'TERYT gminy', 'Liczba wyborców uprawnionych do głosowania', 'Liczba głosów ważnych', 'Liczba głosów nieważnych',
         'Grzegorz Michał Braun', 'Andrzej Sebastian Duda', 'Adam Sebastian Jarubas', 'Bronisław Maria Komorowski', 'Janusz Ryszard Korwin-Mikke',
        'Marian Janusz Kowalski', 'Paweł Piotr Kukiz', 'Magdalena Agnieszka Ogórek', 'Janusz Marian Palikot', 'Paweł Jan Tanajno', 'Jacek Wilk']]
df.head()

,Województwo,Gmina,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,dolnośląskie,m. Bolesławiec,020101,1542,667,1,11,212,9,215,27,2,163,14,11,0,3
1,dolnośląskie,m. Bolesławiec,020101,1369,728,5,7,203,7,250,24,5,196,23,10,0,3
2,dolnośląskie,m. Bolesławiec,020101,1510,742,8,13,199,7,251,26,3,212,21,5,2,3
3,dolnośląskie,m. Bolesławiec,020101,1523,773,8,8,209,3,303,33,6,186,17,6,0,2
4,dolnośląskie,m. Bolesławiec,020101,1492,701,6,6,192,5,278,28,6,146,23,12,2,3


As I mentioned, I want to sum all the wards in a municipality into a single row.

In [6]:
df = df.groupby(df['TERYT gminy']).sum(numeric_only=True).reset_index()
df.head()

,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,020101,31535,15039,110,152,4018,114,6059,576,68,3336,405,224,19,68
1,020102,10951,4751,26,53,1489,59,1678,180,21,1076,121,50,11,13
2,020103,4539,1943,22,13,776,16,669,50,4,338,45,19,1,12
3,020104,11934,4463,41,41,1680,50,1289,152,39,1074,80,38,5,15
4,020105,5467,2223,14,17,570,28,803,85,14,580,76,38,2,10


I lost the state and municipality names but it doesn't matter since they'll be in a shape file, to be joined by the municipality code.

The capital city of Warsaw has been divided into multiple municipalities but the shape file has only 1 attribute for the capital. It would therefore make sense to aggregate the data from the different parts of Warsaw into one row. 

The municipality codes of different parts of Warsaw are 146502 through 146519.

In [7]:
capital_codes = [str(x) for x in list(range(146502, 146520))]
capital_mask = df['TERYT gminy'].isin(capital_codes)
df['is_capital'] = capital_mask

In [8]:
aggregated_data = df.groupby('is_capital').sum(numeric_only=True).reset_index(drop=True)

In [9]:
aggregated_data

,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,29335344,14082443,115838,115091,4954702,234040,4678225,460291,73468,2950572,335754,191687,25842,62771
1,1353226,816491,9114,9041,224390,4721,352835,25793,4162,148507,18129,19555,3943,5415


The second row represents the votes in Warsaw and the first one represents the votes in the rest of the country. I'll add the second row to the dataframe and remove the first.

In [10]:
aggregated_data=aggregated_data.drop([0])
aggregated_data['TERYT gminy']='146501'
aggregated_data
df = df.drop('is_capital', axis=1)

In [11]:
df = pandas.concat([aggregated_data, df], ignore_index=True).drop_duplicates() 
col = df.pop("TERYT gminy")
df.insert(0, col.name, col)
df.head()

,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,146501,1353226,816491,9114,9041,224390,4721,352835,25793,4162,148507,18129,19555,3943,5415
1,020101,31535,15039,110,152,4018,114,6059,576,68,3336,405,224,19,68
2,020102,10951,4751,26,53,1489,59,1678,180,21,1076,121,50,11,13
3,020103,4539,1943,22,13,776,16,669,50,4,338,45,19,1,12
4,020104,11934,4463,41,41,1680,50,1289,152,39,1074,80,38,5,15


Also, it would be better if the vote data was represented as percentage rather than absolute number of votes. 

In [12]:
df['Grzegorz Michał Braun']=round(df['Grzegorz Michał Braun']/df['Liczba głosów ważnych']*100, 2)
df['Andrzej Sebastian Duda']=round(df['Andrzej Sebastian Duda']/df['Liczba głosów ważnych']*100, 2)
df['Adam Sebastian Jarubas']=round(df['Adam Sebastian Jarubas']/df['Liczba głosów ważnych']*100, 2)
df['Bronisław Maria Komorowski']=round(df['Bronisław Maria Komorowski']/df['Liczba głosów ważnych']*100, 2)
df['Janusz Ryszard Korwin-Mikke']=round(df['Janusz Ryszard Korwin-Mikke']/df['Liczba głosów ważnych']*100, 2)
df['Marian Janusz Kowalski']=round(df['Marian Janusz Kowalski']/df['Liczba głosów ważnych']*100, 2)
df['Paweł Piotr Kukiz']=round(df['Paweł Piotr Kukiz']/df['Liczba głosów ważnych']*100, 2)
df['Magdalena Agnieszka Ogórek']=round(df['Magdalena Agnieszka Ogórek']/df['Liczba głosów ważnych']*100, 2)
df['Janusz Marian Palikot']=round(df['Janusz Marian Palikot']/df['Liczba głosów ważnych']*100, 2)
df['Paweł Jan Tanajno']=round(df['Paweł Jan Tanajno']/df['Liczba głosów ważnych']*100, 2)
df['Jacek Wilk']=round(df['Jacek Wilk']/df['Liczba głosów ważnych']*100, 2)
df.head()

,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,146501,1353226,816491,9114,1.11,27.48,0.58,43.21,3.16,0.51,18.19,2.22,2.40,0.48,0.66
1,020101,31535,15039,110,1.01,26.72,0.76,40.29,3.83,0.45,22.18,2.69,1.49,0.13,0.45
2,020102,10951,4751,26,1.12,31.34,1.24,35.32,3.79,0.44,22.65,2.55,1.05,0.23,0.27
3,020103,4539,1943,22,0.67,39.94,0.82,34.43,2.57,0.21,17.40,2.32,0.98,0.05,0.62
4,020104,11934,4463,41,0.92,37.64,1.12,28.88,3.41,0.87,24.06,1.79,0.85,0.11,0.34


I'll also add a turnout column by suming valid and invalid ballots and dividing by the number of people eligible to vote.

In [13]:
df.insert(3, 'Turnout', round(df[['Liczba głosów ważnych','Liczba głosów nieważnych']].sum(axis=1)/df['Liczba wyborców uprawnionych do głosowania'], 4))
df.head()

,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Turnout,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,146501,1353226,816491,0.6101,9114,1.11,27.48,0.58,43.21,3.16,0.51,18.19,2.22,2.40,0.48,0.66
1,020101,31535,15039,0.4804,110,1.01,26.72,0.76,40.29,3.83,0.45,22.18,2.69,1.49,0.13,0.45
2,020102,10951,4751,0.4362,26,1.12,31.34,1.24,35.32,3.79,0.44,22.65,2.55,1.05,0.23,0.27
3,020103,4539,1943,0.4329,22,0.67,39.94,0.82,34.43,2.57,0.21,17.40,2.32,0.98,0.05,0.62
4,020104,11934,4463,0.3774,41,0.92,37.64,1.12,28.88,3.41,0.87,24.06,1.79,0.85,0.11,0.34


I'll also multiply the turnout by 100 so that it is expressed in percentage.

In [14]:
df['Turnout'] *= 100
df.head()

,TERYT gminy,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Turnout,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
0,146501,1353226,816491,61.01,9114,1.11,27.48,0.58,43.21,3.16,0.51,18.19,2.22,2.40,0.48,0.66
1,020101,31535,15039,48.04,110,1.01,26.72,0.76,40.29,3.83,0.45,22.18,2.69,1.49,0.13,0.45
2,020102,10951,4751,43.62,26,1.12,31.34,1.24,35.32,3.79,0.44,22.65,2.55,1.05,0.23,0.27
3,020103,4539,1943,43.29,22,0.67,39.94,0.82,34.43,2.57,0.21,17.40,2.32,0.98,0.05,0.62
4,020104,11934,4463,37.74,41,0.92,37.64,1.12,28.88,3.41,0.87,24.06,1.79,0.85,0.11,0.34


### Loading the geodata

Now that the data looks nicer, it's time to join the geodata to it.

In [15]:
gmina_df = geopandas.read_file(r"..\\subdivision shapes\\Gminy\\Gminy.shp", encoding="utf-8")
powiat_df = geopandas.read_file(r"..\\subdivision shapes\\Powiaty\\Powiaty.shp", encoding="utf-8")
woj_df = geopandas.read_file(r"..\\subdivision shapes\\Wojewodztwa\\Województwa.shp", encoding="utf-8")

In [16]:
gmina_df.sort_values("JPT_KOD_JE")

,JPT_SJR_KO,JPT_KOD_JE,JPT_NAZWA_,JPT_ORGAN_,JPT_JOR_ID,WERSJA_OD,WERSJA_DO,WAZNY_OD,WAZNY_DO,JPT_KOD__1,...,JPT_KJ_I_1,JPT_KJ_I_2,JPT_OPIS,JPT_SPS_KO,ID_BUFOR_1,JPT_ID,JPT_KJ_I_3,Shape_Leng,Shape_Area,geometry
565,GMI,0201011,Bolesławiec,NaN,13381,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,0201011,NaN,NaN,UZG,0,827797,NaN,0.449042,0.002943,"POLYGON ((15.56824 51.22638, 15.55446 51.23078..."
863,GMI,0201022,Bolesławiec,NaN,13381,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,0201022,NaN,NaN,UZG,0,827796,NaN,1.857742,0.037260,"POLYGON ((15.66514 51.17560, 15.65212 51.17788..."
1814,GMI,0201032,Gromadka,NaN,13381,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,0201032,NaN,NaN,UZG,0,827798,NaN,1.111171,0.034588,"POLYGON ((15.77739 51.28384, 15.76557 51.29066..."
865,GMI,0201043,Nowogrodziec,NaN,13381,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,0201043,NaN,NaN,UZG,0,827800,NaN,1.102200,0.022688,"POLYGON ((15.49663 51.17975, 15.49246 51.17540..."
566,GMI,0201052,Osiecznica,NaN,13381,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,0201052,NaN,NaN,UZG,0,827802,NaN,1.401947,0.056439,"POLYGON ((15.33974 51.25966, 15.32596 51.26488..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,GMI,3218043,Resko,NaN,13154,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,3218043,NaN,NaN,UZG,0,828132,NaN,1.689540,0.038866,"POLYGON ((15.27758 53.67434, 15.26765 53.67886..."
445,GMI,3218053,Węgorzyno,NaN,13154,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,3218053,NaN,NaN,UZG,0,828140,NaN,1.330262,0.034725,"POLYGON ((15.46766 53.47730, 15.46547 53.48218..."
2307,GMI,3261011,Koszalin,NaN,13156,2012-09-26,1899-11-30,2012-09-26,1899-11-30,NaN,...,3261011,NaN,NaN,UZG,0,828142,NaN,0.885867,0.013538,"POLYGON ((16.13673 54.17374, 16.13746 54.17750..."
2177,GMI,3262011,Szczecin,NaN,13157,2018-12-15,1899-11-30,2012-09-26,1899-11-30,NaN,...,3262011,NaN,NaN,UZG,0,1351563,NaN,2.083153,0.040639,"POLYGON ((14.77799 53.40298, 14.78367 53.40206..."


I'll keep the municipality code, name and geometry columns and discard the rest.

In [17]:
gmina_df = gmina_df[['JPT_KOD_JE','JPT_NAZWA_', 'geometry']]
gmina_df.head()

,JPT_KOD_JE,JPT_NAZWA_,geometry
0,2602042,Nagłowice,"POLYGON ((20.07881 50.61226, 20.06774 50.61723..."
1,2602063,Sędziszów,"POLYGON ((19.94997 50.50478, 19.93508 50.50231..."
2,2602012,Imielno,"POLYGON ((20.34121 50.54595, 20.34016 50.54625..."
3,2604012,Bieliny,"POLYGON ((20.88780 50.82310, 20.89121 50.82701..."
4,2604112,Mniów,"POLYGON ((20.50742 50.95794, 20.49880 50.95827..."


In [18]:
gmina_df.to_file('file.shp', driver='ESRI Shapefile')

I also have to remove the last digit from the municipality code so that it's the same as in the dataframe containing elections.

In [19]:
gmina_df['JPT_KOD_JE'] = gmina_df['JPT_KOD_JE'].str[:-1]
gmina_df.head()

,JPT_KOD_JE,JPT_NAZWA_,geometry
0,260204,Nagłowice,"POLYGON ((20.07881 50.61226, 20.06774 50.61723..."
1,260206,Sędziszów,"POLYGON ((19.94997 50.50478, 19.93508 50.50231..."
2,260201,Imielno,"POLYGON ((20.34121 50.54595, 20.34016 50.54625..."
3,260401,Bieliny,"POLYGON ((20.88780 50.82310, 20.89121 50.82701..."
4,260411,Mniów,"POLYGON ((20.50742 50.95794, 20.49880 50.95827..."


In [20]:
powiat_df.head()

,JPT_KOD_JE,JPT_NAZWA_,geometry
0,1807,powiat krośnieński,"POLYGON ((21.89480 49.78109, 21.88301 49.76796..."
1,2804,powiat elbląski,"POLYGON ((19.97906 54.13375, 19.98676 54.11915..."
2,1415,powiat ostrołęcki,"POLYGON ((21.55169 53.47813, 21.56589 53.48095..."
3,0413,powiat sępoleński,"POLYGON ((17.39065 53.49096, 17.39383 53.50781..."
4,3019,powiat pilski,"POLYGON ((16.39053 53.13095, 16.39835 53.14838..."


In [21]:
woj_df.head()

,JPT_NAZWA_,geometry
0,zachodniopomorskie,"POLYGON ((16.69909 54.56925, 16.71161 54.55508..."
1,lubuskie,"POLYGON ((14.97418 51.36395, 14.98362 51.37405..."
2,dolnośląskie,"POLYGON ((16.90790 50.44945, 16.88072 50.43616..."
3,wielkopolskie,"POLYGON ((19.04712 52.33280, 19.03636 52.31534..."
4,pomorskie,"POLYGON ((19.64874 54.45327, 19.63571 54.39224..."


Merging the geodata with the df containing the result of the elections.

In [24]:
merged_df = gmina_df.set_index("JPT_KOD_JE").join(df.set_index("TERYT gminy"))
merged_df.head()

,JPT_NAZWA_,geometry,Liczba wyborców uprawnionych do głosowania,Liczba głosów ważnych,Turnout,Liczba głosów nieważnych,Grzegorz Michał Braun,Andrzej Sebastian Duda,Adam Sebastian Jarubas,Bronisław Maria Komorowski,Janusz Ryszard Korwin-Mikke,Marian Janusz Kowalski,Paweł Piotr Kukiz,Magdalena Agnieszka Ogórek,Janusz Marian Palikot,Paweł Jan Tanajno,Jacek Wilk
JPT_KOD_JE,,,,,,,,,,,,,,,,,
260204,Nagłowice,"POLYGON ((20.07881 50.61226, 20.06774 50.61723...",4194,1591,38.29,15,0.50,45.51,17.41,15.40,2.20,0.19,14.90,2.77,1.01,0.06,0.06
260206,Sędziszów,"POLYGON ((19.94997 50.50478, 19.93508 50.50231...",10551,4821,46.03,36,0.52,41.24,16.45,15.91,2.63,0.21,18.81,2.82,0.95,0.00,0.46
260201,Imielno,"POLYGON ((20.34121 50.54595, 20.34016 50.54625...",3684,1472,40.15,7,0.14,43.68,25.07,15.15,1.22,0.48,11.82,1.90,0.27,0.20,0.07
260401,Bieliny,"POLYGON ((20.88780 50.82310, 20.89121 50.82701...",7892,3525,44.84,14,0.51,51.69,17.82,8.48,3.94,0.40,15.32,0.99,0.40,0.06,0.40
260411,Mniów,"POLYGON ((20.50742 50.95794, 20.49880 50.95827...",7452,3512,47.44,23,0.57,50.00,17.57,11.30,3.08,1.03,14.58,0.97,0.54,0.03,0.34


I'll save the cleaned data to a new file.

In [25]:
merged_df.to_excel("../data/elections2015_clean.xlsx")